In [ ]:
# ---- Parameters ----
SCREEN = "20250216_AWALS37_Full_screen_n96"
ANALYSIS = "binary_loocv"
MARKER = "all"
ANTIBODY = "FUS/EEA1"

In [2]:
import os
import random
import numpy as np
import torch

# ---- Set seeds for reproducibility ----
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
from maps.screens import ImageScreenMultiAntibody
import json
from pathlib import Path


# --- Initialize parameters ---
pdir = Path("/home/kkumbier/als/scripts/pipelines/params")
with open(pdir / f"{ANALYSIS}.json", "r") as f:
    params = json.load(f)

params["screen"] = SCREEN
params["antibodies"] = ANTIBODY

# Update marker if specified
if MARKER != "all":
    fstr = params["preprocess"]["drop_feature_types"]["feature_str"]
    fstr += f"|^.*{MARKER}.*$"
    params["preprocess"]["drop_feature_types"]["feature_str"] = fstr

In [4]:
# Load and process screens for train / test
screen = ImageScreenMultiAntibody(params)
screen.load(antibody=params["antibodies"])

print("Processing data...")
screen.preprocess()
assert screen.data is not None, "Loading failed"

for ab in params["antibodies"]:
    print(f"Marker set: {ab}")
    print(f"Data: {screen.data[ab].shape}")

Processing data...
Preprocessing complete
Marker set: FUS/EEA1
Data: (115816, 315)
Marker set: COX IV/Galectin3/atubulin
Data: (108749, 312)


In [ ]:
from maps.analyses import MAP
map_analysis = MAP(screen)
map_analysis.fit()

In [ ]:
import pickle

ab_string = "_".join(ANTIBODY).replace("/", "-")
output_dir = Path(params.get("result_dir")) / params.get("screen")
with open(output_dir / f"{ANALYSIS}-{ab_string}-{MARKER}.pkl", "wb") as f:
    pickle.dump({"analysis": map_analysis, "params": params}, f)